In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *
from sklearn.model_selection import train_test_split
from rapidfuzz import process, fuzz

In [2]:
test_set_df=pd.read_csv("../../data/test_dataset.csv")
test_set_df

,modelID,category,modelAlias,makerAlias
0,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai
1,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai automation
2,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai co.
3,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai corporation
4,MO736784,RF,p50e 12 n,cosel
...,...,...,...,...
31627325,MO955,PKG ETC,fad2300,musashi shotmaster
31627326,MO955,PKG ETC,fad2300,musasi
31627327,MO955,PKG ETC,fad2300,protec
31627328,MO955,PKG ETC,fad2300,toshiba


In [3]:
test_df=test_set_df[~test_set_df["modelAlias"].isna()]
test_df.drop_duplicates(subset=["category","modelAlias", "makerAlias"],inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df.fillna("", inplace=True)
test_df

/tmp/ipykernel_1240622/4158888836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop_duplicates(subset=["category","modelAlias", "makerAlias"],inplace=True)
/tmp/ipykernel_1240622/4158888836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.fillna("", inplace=True)


,modelID,category,modelAlias,makerAlias
0,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai
1,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai automation
2,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai co.
3,MO737993,Motion Control,rcs2 se7c i 60 16 250 t2 m,iai corporation
4,MO736784,RF,p50e 12 n,cosel
...,...,...,...,...
31617266,MO955,PKG ETC,fad2300,musashi shotmaster
31617267,MO955,PKG ETC,fad2300,musasi
31617268,MO955,PKG ETC,fad2300,protec
31617269,MO955,PKG ETC,fad2300,toshiba


In [4]:
rows_to_drop = []
for index, row in test_df.iterrows():
    query = "unknown"
    target = f"{row['modelAlias']} {row['makerAlias']}"
    score = fuzz.token_sort_ratio(query, target)
    if score >= 70:
        rows_to_drop.append(index)
test_df.drop(index=rows_to_drop)

    

KeyboardInterrupt: 

In [ ]:
reference_tuples = list(zip(test_df["modelID"], test_df["modelAlias"], test_df["makerAlias"]))

In [ ]:
train_df, test_split_df = train_test_split(test_df, test_size=0.01, random_state=42)


In [ ]:

def fuzzy_match_model_id_fast(model_alias, maker_alias, reference_tuples, top_k=10):
    query = f"{model_alias} {maker_alias}"
    choices = [f"{alias} {maker}" for _, alias, maker in reference_tuples]
    ids = [model_id for model_id, _, _ in reference_tuples]

    # Top-k 유사도 결과 가져오기
    matches = process.extract(query, choices, scorer=fuzz.token_sort_ratio, limit=top_k, score_cutoff=80)
    
    # modelID, score 추출
    top_matches = [(ids[idx], score) for (_, score, idx) in matches]
    
    return top_matches


In [12]:
from tqdm import tqdm

results = []

filtered_test_df = test_df.dropna(subset=["modelAlias", "makerAlias"])

for _, row in tqdm(filtered_test_df.iloc[1000:1100].iterrows(), total=len(filtered_test_df.iloc[:100])):
    top_matches = fuzzy_match_model_id_fast(row["modelAlias"], row["makerAlias"], reference_tuples, top_k=5)
    
    # Top-k 결과를 리스트로 저장
    results.append({
        "true_modelID": row["modelID"],
        "top_predictions": [model_id for model_id, _ in top_matches],
        "top_scores": [score for _, score in top_matches],
        "is_correct_in_top_k": row["modelID"] in [model_id for model_id, _ in top_matches]
    })

results_df = pd.DataFrame(results)


  2%|▏         | 2/100 [00:21<17:14, 10.56s/it]


KeyboardInterrupt: 

In [17]:
results_df.top_predictions[0]

['MO1845', 'MO1845', 'MO1818', 'MO1852', 'MO1845']

In [ ]:
reference_df = test_df.dropna(subset=["modelAlias", "makerAlias"]).drop_duplicates(subset=["modelID", "modelAlias", "makerAlias"])
reference_tuples = list(zip(reference_df["modelID"], reference_df["modelAlias"], reference_df["makerAlias"]))


In [13]:
from rapidfuzz import process, fuzz

def fuzzy_match_worker(row):
    model_alias = row["modelAlias"]
    maker_alias = row["makerAlias"]
    true_id = row["modelID"]
    query = f"{model_alias} {maker_alias}"
    
    # 전역 변수 reference_tuples 사용
    choices = [f"{alias} {maker}" for _, alias, maker in reference_tuples]
    ids = [model_id for model_id, _, _ in reference_tuples]

    match, score, index = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
    return {
        "true_modelID": true_id,
        "predicted_modelID": ids[index],
        "score": score
    }


In [15]:
import multiprocessing as mp
from tqdm import tqdm

filtered_rows = test_df[:100].dropna(subset=["modelAlias", "makerAlias"]).to_dict("records")

with mp.Pool(mp.cpu_count()) as pool:
    results = list(tqdm(pool.imap(fuzzy_match_worker, filtered_rows), total=len(filtered_rows)))


  0%|          | 0/100 [02:14<?, ?it/s]


KeyboardInterrupt: 